In [ ]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import paramiko

In [ ]:
# connect to remote sftp server to retrieve file
# credentials removed for security

client = paramiko.SSHClient()
# automatically add keys without requiring human intervention
client.set_missing_host_key_policy( paramiko.AutoAddPolicy() )

client.connect(sftp_url, port=22, username=sftp_user, password=sftp_pass)

sftp = client.open_sftp()
files = sftp.listdir()

# lists all the folders in the directory after connecting through the sftp
print(files)

In [ ]:
# read file to memory from the sftp server
with sftp.open('/home/jsinger/hue-health/healthcare_claims_data.xlsx') as f:
    f.prefetch()
    # we have two sheets in the Excel file; save them as two variables
    claims_data_pre65 = pd.read_excel(f, sheet_name='Pre65')
    claims_data_post65 = pd.read_excel(f, sheet_name='Post65')

In [ ]:
# increase the number of columns displayed in this notebook to 100
pd.options.display.max_columns = 100

In [ ]:
claims_data_pre65.columns == claims_data_post65.columns

In [ ]:
# both sheets contain the same features, so we'll combine them into one dataframe
# first, let's add a column to each df to keep track of the original sheet
claims_data_pre65['sheet_name'] = 'Pre65'
claims_data_post65['sheet_name'] = 'Post65'

# next, we'll combine the two sheets of data into one df and reset the index
claims_data = pd.concat([claims_data_pre65,claims_data_post65], axis=0, ignore_index=True)

In [ ]:
claims_data.head()

In [ ]:
claims_data['sheet_name'].value_counts()

In [ ]:
claims_data.info()

In [ ]:
claims_data.isnull().sum()

In [ ]:
claims_data.dtypes

In [ ]:
# converting LINE_NO and MEMBER_ID columns to object type
claims_data['LINE_NO'] = claims_data['LINE_NO'].astype('object')
claims_data['MEMBER_ID'] = claims_data['MEMBER_ID'].astype('object')

In [ ]:
claims_data.describe(include='all').loc[['unique'],['LINE_NO','CLAIM_ID','MEMBER_ID']]

The 1,052,150 line items (services) account for 477,389 claims across 34,985 members.

## Initial Data Cleaning & Data Quality Check

Let's drop the following columns that have all missing values:
 * Re-Priced\nAmount
 * Provider In\nPPO Network\n(Yes / No)
 * Not Re-Priced\n(Indicate w/ "x")
 * Capitated (Y/N)
 * Capitation Description
 
Let's also drop the following columns that will not be useful for our analysis:
 * PROV_TIN
 * PROV_ADDR1
 * PROV_ADDR2

In [ ]:
# drop columns
drop_columns = ['Re-Priced\nAmount', 'Provider In\nPPO Network\n(Yes / No)', 'Not Re-Priced\n(Indicate w/ "x")',
               'Capitated (Y/N)', 'Capitation Description', 'PROV_TIN', 'PROV_ADDR1', 'PROV_ADDR2']

claims_data.drop(drop_columns, axis=1, inplace=True)

In [ ]:
claims_data.head()

Let's do a quick data quality check on the following features:
 * CLAIM_TYPE: should be either Professional (PR) or Institutional (IN)
 * UB_BILL_TYPE: should be 3 digits/characters. See [here](https://med.noridianmedicare.com/web/jea/topics/claim-submission/bill-types) for reference

In [ ]:
claims_data['CLAIM_TYPE'].value_counts()

In [ ]:
bill_type_wrong_count = 0
for value in claims_data['UB_BILL_TYPE']:
    if np.nan:
        pass
    elif len(value) != 3:
        bill_type_wrong_count += 1
        
bill_type_wrong_count

In [ ]:
claims_data['UB_BILL_TYPE'].unique()

In [ ]:
claims_data.isnull().sum()

## Patient Stratification

Since we'd like to stratify members by number of services provided and total charges, need to restructure data to format where each row represents one member.

In [ ]:
# per member, calculate the number of line items (services), unique claims, and total charges
member_group = claims_data.groupby('MEMBER_ID').aggregate({'LINE_NO': lambda x: len(x.unique()),
                                            'CLAIM_ID': lambda x: len(x.unique()),'Eligible Charges': np.sum})
member_group

In [ ]:
member_group.corr()

In [ ]:
plt.scatter(member_group['LINE_NO'],member_group['CLAIM_ID'])
plt.title('Per member number of line items by number of claims')
plt.xlabel('Number of Line Items')
plt.ylabel('Number of Claims')
plt.show()

Since our data is at the line (service) level rather than the claim level, and the correlation between number of line items and charges is stronger than that between number of claims and charges, we'll use the LINE_NO feature for our model. Further, the per member number of line items correlates strongly with the per member number of claims, so we likely wouldn't see much difference in clusters if grouping at the claim level.

In [ ]:
member_group = claims_data.groupby('MEMBER_ID').aggregate({'LINE_NO': lambda x: len(x.unique()),
                                                           'Eligible Charges': np.sum})
member_group.rename(columns={'LINE_NO':'Num Lines'}, inplace=True)
member_group

In [ ]:
# visualize Num Lines v. Eligible Charges
plt.figure(figsize=(6,6))
plt.scatter(member_group['Num Lines'],member_group['Eligible Charges'])
plt.title('Per member number of line items by total charges')
plt.xlabel('Number of Line Items')
plt.ylabel('Total Charges')
plt.ticklabel_format(style='plain')

plt.show()

In [ ]:
member_group.plot.scatter(x='Num Lines', y='Eligible Charges',alpha=0.3,figsize=(6,6))
plt.ticklabel_format(style='plain')
plt.title('Per member number of line items by total charges')
plt.xlabel('Number of Line Items')
plt.ylabel('Total Charges');

In [ ]:
member_group.describe(percentiles=[0.05,0.1,0.25,0.5,0.75,0.9,0.95])

## K-Means Clustering

Let's use the Elbow Method to get an idea of what a good k number of clusters would be, by evaluating the sum of squared distance (SSE) between data points and their assigned clusters' centroids. 

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
# Run the Kmeans algorithm and get the index of data points clusters
sse = []
list_k = list(range(1, 10))

for k in list_k:
    km = KMeans(n_clusters=k)
    km.fit(member_group[['Num Lines', 'Eligible Charges']])
    sse.append(km.inertia_)

# Plot sse against k
plt.figure(figsize=(6, 6))
plt.plot(list_k, sse, '-o')
plt.xlabel(r'Number of clusters *k*')
plt.ylabel('Sum of squared distance');

Looks like k between 3 and 5 would be reasonable choices for number of clusters. Let's run and visualize kmeans models for k= 3, 4 and 5.

In [ ]:
# fit kmeans model
def kmeans_model(k):
    km = KMeans(n_clusters=k, random_state=1)
    km.fit(member_group[['Num Lines', 'Eligible Charges']])
    return km

In [ ]:
km_3 = kmeans_model(3)
km_4 = kmeans_model(4)
km_5 = kmeans_model(5)

In [ ]:
# add columns with cluster labels to the member_group dataframe
member_group['cluster_3'] = km_3.labels_
member_group['cluster_4'] = km_4.labels_
member_group['cluster_5'] = km_5.labels_

In [ ]:
member_group.head()

In [ ]:
member_group['cluster_3'].value_counts(normalize=True)*100

In [ ]:
member_group['cluster_4'].value_counts(normalize=True)*100

In [ ]:
member_group['cluster_5'].value_counts(normalize=True)*100

In [ ]:
# Visualizing clusters
def visualize_clusters(df, num_clusters, col_name, centroids):
    colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k']

    for n in range(num_clusters):
        clustered_df = df[df[col_name] == n]
        plt.scatter(clustered_df['Num Lines'], clustered_df['Eligible Charges'], c=colors[n-1])
        plt.scatter(centroids[:,0], centroids[:,1], c='y', marker='*', s=100)
        plt.xlabel('Number of Line Items', fontsize=13)
        plt.ylabel('Total Charges', fontsize=13)
        plt.ticklabel_format(style='plain')
    plt.show()

In [ ]:
visualize_clusters(member_group,3,'cluster_3',km_3.cluster_centers_)

In [ ]:
visualize_clusters(member_group,4,'cluster_4',km_4.cluster_centers_)

In [ ]:
visualize_clusters(member_group,5,'cluster_5',km_5.cluster_centers_)

In [ ]:
# centroids for each cluster for km_4
np.set_printoptions(suppress=True) # suppress scientific notation
km_4.cluster_centers_

Since we'll use the kmeans model with 4 clusters, let's drop the cluster_3 and cluster_5 columns from our df, and rename the column to 'cluster'.

In [ ]:
member_group_cluster = member_group.drop(['cluster_3','cluster_5'],axis=1)
member_group_cluster.rename(columns={'cluster_4':'cluster'},inplace=True)
member_group_cluster

In [ ]:
member_group_cluster['cluster'].value_counts(normalize=True)*100

The clusters generated by the km_4 model don't appear to align with our healthcare-specific expectations of the following groups of patients:
 * Low cost, low utilizers
 * Low cost, high utilizers
 * High cost, low utilizers
 * High cost, high utilizers
 
So, we'll manually stratify our patient population by setting bounds we think are more appropriate.

## Manual Stratification

In [ ]:
member_group_manual = member_group_cluster.drop('cluster',axis=1)
member_group_manual.head()

In [ ]:
member_group_manual.describe(percentiles=[.10,.25,.50,.75,.90])

We'll consider low cost to be less than average ($1,710) and low utilization to be less than average (12)

In [ ]:
# low cost, low utilizers (lclu) - 25% for # services and charges
lclu = (member_group_manual['Num Lines']<=12) & (member_group_manual['Eligible Charges']<=1710)

# high cost, low utilizers (hclu) - >75% for charges, 25% for # services
hclu = (member_group_manual['Num Lines']<=12) & (member_group_manual['Eligible Charges']>1710)

# low cost, high utilizers (lchu) - 25% for charges, 75% for # services
lchu = (member_group_manual['Num Lines']>12) & (member_group_manual['Eligible Charges']<=1710)

# high cost, high utilizers (hchu) - 75% for charges and # services
hchu = (member_group_manual['Num Lines']>12) & (member_group_manual['Eligible Charges']>1710)

Let's check to make sure we capture all members

In [ ]:
member_group_manual.shape[0]

In [ ]:
member_group_manual[lclu].shape[0] + member_group_manual[hclu].shape[0] + member_group_manual[lchu].shape[0] + member_group_manual[hchu].shape[0]

In [ ]:
member_group_manual.head()

In [ ]:
# add columns with our group labels
member_group_manual_lclu = member_group_manual[lclu].copy()
member_group_manual_lclu['group'] = 'lclu'

member_group_manual_hclu = member_group_manual[hclu].copy()
member_group_manual_hclu['group'] = 'hclu'

member_group_manual_lchu = member_group_manual[lchu].copy()
member_group_manual_lchu['group'] = 'lchu'

member_group_manual_hchu = member_group_manual[hchu].copy()
member_group_manual_hchu['group'] = 'hchu'

In [ ]:
# combine group dfs
member_group_manual = pd.concat([member_group_manual_lclu,member_group_manual_hclu,
                                member_group_manual_lchu,member_group_manual_hchu])

In [ ]:
member_group_manual.head()

In [ ]:
# convert group to category
member_group_manual['group'] = member_group_manual['group'].astype('category')

In [ ]:
member_group_manual.dtypes

In [ ]:
# visualize our groups
# fig, ax = plt.subplots(figsize=(6,6))
colors = {'lclu':'green','hclu':'yellow','lchu':'blue','hchu':'red'}
# #member_group_manual.plot.scatter('Num Lines', 'Eligible Charges', c=member_group_manual['group'].map(colors), figsize=(10,10));
# ax.scatter(member_group_manual['Num Lines'], member_group_manual['Eligible Charges'], c=member_group_manual['group'].map(colors))
# ax.ticklabel_format(style='plain')
# plt.show()

fig = plt.figure(figsize=(15,6))

ax1 = fig.add_subplot(121)
ax1.scatter(member_group_manual['Num Lines'], member_group_manual['Eligible Charges'], c=member_group_manual['group'].map(colors))
ax1.ticklabel_format(style='plain')

ax2 = fig.add_subplot(122)
ax2.scatter(member_group_manual['Num Lines'], member_group_manual['Eligible Charges'], c=member_group_manual['group'].map(colors))
ax2.axis([0,500,0,500000])

plt.show()


In [ ]:
member_group_manual['group'].value_counts()

In [ ]:
member_group_manual['group'].value_counts(normalize=True)*100

In [ ]:
plt.scatter(member_group_manual_lclu['Num Lines'], member_group_manual_lclu['Eligible Charges']);

In [ ]:
plt.figure(figsize=(6,6))
plt.scatter(member_group_manual['Num Lines'], member_group_manual['Eligible Charges'])
plt.axvline(12,c='r')
plt.axhline(1710,c='r')
plt.ticklabel_format(style='plain')
plt.axis([0,500,0,100000])
plt.show()

In [ ]:
# % of members with more than 100 lines
(member_group_manual['Num Lines']>100).sum()/len(member_group_manual)

In [ ]:
# % of members with more than 100 lines
(member_group_manual['Eligible Charges']>500000).sum()/len(member_group_manual)

## Analysis

In [ ]:
member_group_manual['group'].value_counts()

In [ ]:
member_group_manual

In [ ]:
# get list of members for each group
lclu_members = list(member_group_manual[lclu].index)
hclu_members = list(member_group_manual[hclu].index)
lchu_members = list(member_group_manual[lchu].index)
hchu_members = list(member_group_manual[hchu].index)
print(lclu_members[:10])
print(hclu_members[:10])
print(lchu_members[:10])
print(hchu_members[:10])

In [ ]:
# create dataframes with line items for each member group
lclu_claims = claims_data[claims_data['MEMBER_ID'].isin(lclu_members)]
hclu_claims = claims_data[claims_data['MEMBER_ID'].isin(hclu_members)]
lchu_claims = claims_data[claims_data['MEMBER_ID'].isin(lchu_members)]
hchu_claims = claims_data[claims_data['MEMBER_ID'].isin(hchu_members)]

Find each group's % of total services and total cost

In [ ]:
# calculate total # line items and charges for the entire patient population
print(claims_data.shape[0])
print(claims_data['Eligible Charges'].sum())

In [ ]:
# calculate each groups # line items and charges
groups = [lclu_claims, hclu_claims, lchu_claims, hchu_claims]
group_items = {}
group_charges = {}
for i,group in enumerate(groups):
    group_items[i] = group.shape[0]
    group_charges[i] = group['Eligible Charges'].sum()

In [ ]:
group_items

In [ ]:
group_charges

In [ ]:
# calculate % of total items
group_items_pct = {}
for key,value in group_items.items():
    group_items_pct[key] = value / claims_data.shape[0]

In [ ]:
# calculate % of total charges
group_charge_pct = {}
for key,value in group_charges.items():
    group_charge_pct[key] = value / claims_data['Eligible Charges'].sum()

In [ ]:
print(group_items_pct)
print(group_charge_pct)

Quick dive into High cost, low utilizers - we'd expect charges to reflect either traumatic injuries or serious conditions that require infrequent, but expensive treatment. Much of these costs may be unpreventable.

In [ ]:
hclu_charges = hclu_claims[['MEMBER_ID','CLAIM_ID','PROC_CD','DRG_CODE','DIAG1_CD','Eligible Charges']].sort_values('Eligible Charges',ascending=False)
hclu_charges

In [ ]:
# filter for highest charges
hclu_charges.head(20)

Find the most common CPT/Dx code combinations for Groups: HCLU, LCHU, HCHU

In [ ]:
# group 2: HCLU
hclu_cpt_dx_combs = hclu_claims.groupby(['PROC_CD','DIAG1_CD']).size().sort_values(ascending=False)
hclu_cpt_dx_combs.head(20).sort_values(ascending=True).plot.barh(figsize=(6,6))
plt.title('High Cost, Low Utilizers - Top 20 CPT/DX Code Combinations')
plt.xlabel('Count')
plt.ylabel('CPT, DX')
plt.show()

In [ ]:
# for HCLU patients, we're more interested in the highest cost services, not just the most common
# so let's filter the top code combinations for those with associated average charges above the mean

# first, we'll find the average service charge in the hclu claims group
hclu_claims['Eligible Charges'].mean()

In [ ]:
# then, we'll filter our code combinations
hclu_claims_high_cost = hclu_claims[hclu_claims['Eligible Charges']>450]
hclu_cpt_dx_combs_high_cost = hclu_claims_high_cost.groupby(['PROC_CD','DIAG1_CD']).size().sort_values(ascending=False)
hclu_cpt_dx_combs_high_cost.head(20).sort_values(ascending=True).plot.barh(figsize=(6,6))
plt.title('HCLU Top 20 CPT/DX Code Combinations, Charges > 50%')
plt.xlabel('Count')
plt.ylabel('CPT, DX')
plt.show()

In [ ]:
# looks like there's high variability with the 77067 charge
cpt_77067_lines = claims_data[claims_data['PROC_CD']==77067]
cpt_77067_lines['Eligible Charges'].describe()

Indeed, std = 154.28, and the charge can range from 38.75 to 1,337.51. Let's see if there's any correlation with cost of cpt 77067 and other variables.

In [ ]:
cpt_77067_lines.corr()['Eligible Charges']

Looks like there's a strong correlation with a few different variables: PROV_NPI, PROV_ZIP, MEMBER_ZIP, and SERVICE_COUNT

In [ ]:
len(cpt_77067_lines['PROV_CITY'].unique())

In [ ]:
cpt_77067_lines.groupby('PROV_CITY')['Eligible Charges'].mean()

In [ ]:
cpt_77067_lines.groupby('PROV_ST')['Eligible Charges'].mean().sort_values().plot.barh(figsize=(8,10))
plt.title('Average Charge for CPT 77067 by State')
plt.xlabel('Mean Charge')
plt.ylabel('Provider State')
plt.show()

In [ ]:
cpt_77067_lines.groupby('PROV_NAME')['Eligible Charges'].mean().sort_values(ascending=False)

In [ ]:
# filter for providers in CO and WY
cpt_77067_lines_WY_CO = cpt_77067_lines[(cpt_77067_lines['PROV_ST']=='WY') | (cpt_77067_lines['PROV_ST']=='CO')]
cpt_77067_lines_WY_CO.groupby('PROV_NAME')['Eligible Charges'].mean().sort_values(ascending=False)

In [ ]:
cpt_77067_lines_WY_CO.corr()['Eligible Charges']

In [ ]:
cpt_77067_lines_WY_CO.groupby('PLACE_OF_SERVICE').mean()

In [ ]:
ax = cpt_77067_lines_WY_CO.groupby('PLACE_OF_SERVICE').mean()['Eligible Charges'].sort_values(ascending=False).plot.bar(rot=0)
ax.set(xlabel='Place of Service', ylabel='Average Charge ($)', title='Average Charge for CPT 77067 by Place of Service (WY & CO)');

In [ ]:
# group 3: LCHU
lchu_cpt_dx_combs = lchu_claims.groupby(['PROC_CD','DIAG1_CD']).size().sort_values(ascending=False)
lchu_cpt_dx_combs.head(20).sort_values(ascending=True).plot.barh(figsize=(6,6))
plt.title('Low Cost, High Utilizers - Top 20 CPT/DX Code Combinations')
plt.xlabel('Count')
plt.ylabel('CPT, DX')
plt.show()

In [ ]:
# group 3: HCHU
hchu_cpt_dx_combs = hchu_claims.groupby(['PROC_CD','DIAG1_CD']).size().sort_values(ascending=False)
hchu_cpt_dx_combs.head(20).sort_values(ascending=True).plot.barh(figsize=(6,6))
plt.title('High Cost, High Utilizers - Top 20 CPT/DX Code Combinations')
plt.xlabel('Count')
plt.ylabel('CPT, DX')
plt.show()

In [ ]:
hchu_claims['Eligible Charges'].mean()

In [ ]:
hchu_claims_high_cost = hchu_claims[hchu_claims['Eligible Charges']>512]
hchu_cpt_dx_combs_high_cost = hchu_claims_high_cost.groupby(['PROC_CD','DIAG1_CD']).size().sort_values(ascending=False)
hchu_cpt_dx_combs_high_cost.head(20).sort_values(ascending=True).plot.barh(figsize=(6,6))
plt.title('HCHU Top 20 CPT/DX Code Combinations, Charges > 50%')
plt.xlabel('Count')
plt.ylabel('CPT, DX')
plt.show()

In [ ]:
cpt_98941_lines = claims_data[claims_data['PROC_CD']==98941]
cpt_98941_lines['Eligible Charges'].describe()

In [ ]:
cpt_98941_lines.corr()['Eligible Charges']

In [ ]:
cpt_98941_lines.groupby('PROV_NAME')['Eligible Charges'].mean().sort_values(ascending=False)

In [ ]:
cpt_98941_lines.groupby('DRG_CODE')['Eligible Charges'].mean().sort_values(ascending=False)

In [ ]:
ax = cpt_98941_lines.groupby('DRG_CODE')['Eligible Charges'].mean().sort_values(ascending=False).plot.bar(rot=0)
ax.set(xlabel='DRG Code', ylabel='Average Charge ($)', title='Average Charge for CPT 98941 by DRG Code')
ax.text(-0.15,129,'126.39')
ax.text(0.85,70,'68.00')
ax.text(1.85,61,'58.79')
ax.set_ylim([0,140]);